# The HFM library - A fast marching solver with adaptive stencils

## Part : Seismology
## Chapter : Tilted transversally isotropic metrics

In this notebook, we demonstrate anisotropic fast marching with a class of metrics arising in seismic traveltime tomography. 
The intended use cases are fairly similar to [metrics defined by a Hooke tensor](Seismic.ipynb), which illustrates a closely related model. Under the hood, however, we use a completely different implementation.

**Tilted transversally isotropic models (dimension 2).**
We consider eikonal equations of the following form 
$$
    a X^2+b Y^2 + c X^4+d X^2 Y^2 + e Y^4 = 1
$$
in two dimensions, where $a,b,c,d,e$ are real coefficients, and where 
$$
    (X,Y) = A \nabla u
$$
for some linear transformation $A$.

Some algebraic conditions are required on $a,b,c,d,e$ for this equation to make sense. On the other hand, the linear map $A$ is arbitrary.
In the special case where $c=d=e=0$, one recovers a Riemannian eikonal equation.

The numerical scheme is based on rewriting this equation as an extremum of a family of Riemannian eikonal equations, in the form
$$
    \max_{t \in [0,1]} \|\nabla u\|_{D(t)} = 1, 
$$
where $D(t)$ depends on the parameters $a,b,c,d,e,A$ in addition to $t$. From this point, one can rely on the Eulerian discretization of [Riemannian eikonal equations](../Riemannian.ipynb).

**Tilted transversally isotropic models (dimension 3).**
The model is similar up to the insertion of $Y^2+Z^2$, as follows:
$$
    a X^2+b (Y^2+Z^2) + c X^4+d X^2 (Y^2+Z^2) + e (Y^2+Z^2)^2 = 1,
$$
where $a,b,c,d,e$ are again real coefficients, and $(X,Y,Z)$ is again the image of $\nabla u$ by a linear map.

## 0. Importing the required libraries

In [1]:
import sys; sys.path.insert(0,"../..") # Allow import of agd from parent directory (useless if conda package installed)
#from Miscellaneous import TocTools; print(TocTools.displayTOC('Seismic','FMM'))

In [2]:
from agd import HFMUtils
from agd import LinearParallel as lp
from agd.Metrics.Seismic import Hooke,Reduced
from agd import AutomaticDifferentiation as ad
from agd.Plotting import savefig; #savefig.dirName = 'Images/Seismic'
norm_infinity = ad.Optimization.norm_infinity

In [3]:
import numpy as np
#import scipy.linalg
#from copy import copy
%matplotlib inline
import matplotlib.pyplot as plt

## 1. Constant medium

In [4]:
hfmIn_Constant = HFMUtils.dictIn({
    'model':'TTI2',
    'arrayOrdering':'RowMajor',
    'exportValues':1,
    'seeds':[[0.,0.]],
    'factoringMethod':'Static',
    'factoringRadius':10,
#    'tips':[[x,y] for y in HFMUtils.CenteredLinspace(-1,1,6) 
#                    for x in HFMUtils.CenteredLinspace(-1,1,6)],
#    'exportGeodesicFlow':1,
})

hfmIn_Constant.SetRect(sides=[[-1,1],[-1,1]],dimx=5,sampleBoundary=True) # Define the domain
X = hfmIn_Constant.Grid() # Horizontal and vertical axis

metric = Reduced([1.,1],[[0.5,0],[0,0]])
hfmIn_Constant['metric'] = metric

In [5]:
#metric.niter_sqp=20
v=np.array((1.,0.))
grad = metric.gradient((1.,0.))
lvl = metric._dual_level(grad)
print(f"v={v},grad={grad},lvl={lvl}")

v=[1. 0.],grad=[0.85559968 0.        ],lvl=0.0


In [69]:
metric.norm(np.array((1.,0.)))

0.10101370804120646

In [70]:
metric.to_HFM()

array([1.  , 1.  , 1.98, 0.  , 0.  , 1.  , 0.  , 0.  , 1.  ])

In [27]:
hfmOut = hfmIn_Constant.RunSmart()

Field verbosity defaults to 1
Field order defaults to 1
Field spreadSeeds defaults to 0
Field showProgress defaults to 0
Field factoringPointChoice defaults to Key
In RunOnce"accepted.index" -> {2,2},"offsets" -> {{-1,0},{-1,1},{0,-1},{0,1},{1,-1},{1,0}},"acceptedFlags" -> {0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0},
In ConditionalUpdate"updated.index" -> {3,2},"values[updated.linear]" -> inf,"acceptedValue" -> 0,"activeNeighs[updated.linear].none()" -> 1,
In Update"updated.index" -> {3,2},
In HopfLaxUpdate"updated" -> {{3,2},17},"acceptedOffset" -> {-1,0},"value" -> 0,"acceptedPos" -> 1,"norm" -> {"linear" -> {1,1},"quadratic" -> {{2,0},{0,0}},"transform" -> {{2,0},{0,2}},},
"offset" -> {1,-1},"valp" -> -inf,"valm" -> -inf,
"offset" -> {-1,0},"valp" -> 0,"valm" -> -inf,
"offset" -> {0,1},"valp" -> -inf,"valm" -> -inf,
"values" -> {inf,0,inf},
"fm.values" -> {inf,inf,inf,inf,inf,inf,inf,inf,inf,inf,inf,inf,0,inf,inf,inf,inf,inf,inf,inf,inf,inf,inf,inf,inf},
In Update Value "t" 

In [28]:
hfmOut

{'FMCPUTime': 0.018535,
 'GeodesicCPUTime': 5e-06,
 'MaxStencilWidth': 2.0,
 'StencilCPUTime': 0.000212,
 'defaulted': 'exportActiveNeighs exportActiveOffsets exportFactoringCenters exportFactoringRegion exportGeodesicFlow factoringPointChoice order refineStencilAtWallBoundary showProgress spreadSeeds verbosity',
 'nAccepted': 25.0,
 'spreadedSeedValues': array([0.]),
 'spreadedSeeds': array([[0., 0.]]),
 'unusedFromCompute': 'FMCPUTime GeodesicCPUTime MaxStencilWidth StencilCPUTime nAccepted spreadedSeedValues spreadedSeeds values',
 'values': array([[1.5044364 , 1.10721681, 1.        , 1.10721681, 1.5044364 ],
        [1.27640528, 0.7752271 , 0.5       , 0.7752271 , 1.27640528],
        [1.1755705 , 0.58778525, 0.        , 0.58778525, 1.1755705 ],
        [1.27640528, 0.7752271 , 0.5       , 0.7752271 , 1.27640528],
        [1.5044364 , 1.10721681, 1.        , 1.10721681, 1.5044364 ]]),
 'visitedUnset': 'activeNeighs costVariation euclideanScale factoringPoints factoringRegion getSte